In [1]:
import pandas as pd

### (1) 결측값 대체,  `.fillna()` 사용 예시

In [2]:
df = pd.DataFrame(['Korea', 'Japan', float('nan')])
df

,0
0,Korea
1,Japan
2,NaN


In [3]:
df.fillna('Unknown')

,0
0,Korea
1,Japan
2,Unknown


### (2) 범주형 데이터를 수치형으로 변환, `LabelEncoder` 사용 예시 

In [4]:
# 코드 1-14. LabelEncoder 사용 예시

# 6명 선수의 국적
countries = ['Korea', 'Korea', 'China', 'Korea', 'Japan', 'China']

# LabelEncoder 불러오기
from sklearn.preprocessing import LabelEncoder

# LabelEncoder object 생성
lb = LabelEncoder()
# .fit()을 통해 범주형 데이터 고유값을 저장
lb.fit(countries)
# .transform()을 통해 수치형 데이터로 변환
lb.transform(countries)

array([2, 2, 0, 2, 1, 0])

In [5]:
# 코드 1-15. LabelEncoder에 저장된 고유값

lb.classes_

array(['China', 'Japan', 'Korea'],
      dtype='<U5')

In [6]:
# 코드 1-16. LabelEncoder에 다른 크기의 데이터 입력

semi_finalists = ['China', 'China', 'China', 'Korea']
lb.transform(semi_finalists)

array([0, 0, 0, 2])

In [7]:
# 코드 1-17. LabelEncoder에서 새로운 고유값은 변환 불가

finalists = ['China', 'United States', 'China', 'Korea']
lb.transform(finalists)

ValueError: y contains new labels: ['United States']

### 전처리 실행 코드

In [8]:
# 코드 1-18. 전처리 준비

trn = pd.read_csv('data/train_ver2.csv')
tst = pd.read_csv('data/test_ver2.csv')

def check_unique_vals(trn, tst, col):
    print('col   : ', col)
    print('train : ', set(trn[col]))
    print('test  : ', set(tst[col]))
    print('non-overlap : ', set(trn[col]).symmetric_difference(set(tst[col])))
    
def preprocess(trn, tst, col, replace_nan='unk'):
    # 'unk'로 결측값 대체
    trn[col].fillna(replace_nan, inplace=True)
    tst[col].fillna(replace_nan, inplace=True)

    # LabelEncoder로 수치형 데이터로 변환
    lb = LabelEncoder()
    lb.fit(pd.concat([trn[col],tst[col]]))
    trn[col] = lb.transform(trn[col])
    tst[col] = lb.transform(tst[col])
    return trn, tst

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# 코드 1-19. fecha_dato 고유값 출력

col = 'fecha_dato'
check_unique_vals(trn, tst, col)

col   :  fecha_dato
train :  {'2016-01-28', '2015-02-28', '2015-07-28', '2015-12-28', '2016-02-28', '2015-03-28', '2015-09-28', '2015-10-28', '2015-05-28', '2016-03-28', '2015-01-28', '2015-04-28', '2015-06-28', '2016-04-28', '2015-11-28', '2015-08-28', '2016-05-28'}
test  :  {'2016-06-28'}
non-overlap :  {'2016-01-28', '2015-02-28', '2015-07-28', '2015-12-28', '2016-02-28', '2015-03-28', '2015-09-28', '2015-10-28', '2016-06-28', '2015-05-28', '2016-03-28', '2015-01-28', '2015-04-28', '2015-06-28', '2016-04-28', '2015-11-28', '2015-08-28', '2016-05-28'}


In [10]:
# 코드 1-20. ind_empleado 고유값 출력

col = 'ind_empleado'
check_unique_vals(trn, tst, col)

col   :  ind_empleado
train :  {nan, 'F', 'S', 'B', 'N', 'A'}
test  :  {'F', 'S', 'B', 'N', 'A'}
non-overlap :  {nan}


In [11]:
# 코드 1-21. ind_empleado 전처리 결과

trn, tst = preprocess(trn, tst, col)
check_unique_vals(trn, tst, col)

col   :  ind_empleado
train :  {0, 1, 2, 3, 4, 5}
test  :  {0, 1, 2, 3, 4}
non-overlap :  {5}


In [12]:
# 코드 1-22. 전처리 코드

# 1) 범주형 -> 수치형으로 변환하는 변수
cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', \
        'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']
for col in cols:
    print(col)
    trn, tst = preprocess(trn, tst, col)

    
# 2) 날짜 변수        
col = 'fecha_dato'
trn[col] = pd.to_datetime(trn[col])

# 결측값은 첫 계약을 체결한 날짜가 짧다고 가정하고, '2016-06-01'로 변환
col = 'fecha_alta'
trn[col].fillna('2016-06-01', inplace=True)
trn[col] = pd.to_datetime(trn[col])
tst[col] = pd.to_datetime(tst[col])

# 결측값을 데이터에서 가장 최신 날짜로 대체
col = 'ult_fec_cli_1t'
trn[col].fillna('2015-06-30', inplace=True)
tst[col].fillna('2016-05-30', inplace=True)
trn[col] = pd.to_datetime(trn[col])
tst[col] = pd.to_datetime(tst[col])


# 3) 소수형에서 정수형으로 변환하는 변수
col = 'ind_nuevo'
trn[col].fillna(-1,inplace=True)
trn[col] = trn[col].astype(int)

col = 'indrel'
trn[col].fillna(0, inplace=True)
trn[col].replace(99,2, inplace=True)
trn[col] = trn[col].astype(int)
tst[col].replace(99,2, inplace=True)

col = 'age'
# 결측값을 의미하는 ' NA'를 0으로 변환
nan_value = ' NA'
trn[col].replace(nan_value, 0, inplace=True)
# trn 데이터를 정수형 (int)로 변환
trn[col] = trn[col].astype(int)

col = 'antiguedad'
# 결측값을 의미하는 '     NA'를 0으로 변환
nan_value = '     NA'
trn[col].replace(nan_value, 0, inplace=True)
# trn 데이터를 정수형 (int)로 변환
trn[col] = trn[col].astype(int)

col = 'indrel_1mes'
# P를 0으로 대체 후, 데이터를 정수형으로 변환
trn[col].fillna(-1, inplace=True)
tst[col].fillna(-1, inplace=True)
trn[col].replace('P', 0, inplace=True)
trn[col] = trn[col].astype(float).astype(int)
tst[col] = tst[col].astype(int)

col = 'cod_prov'
trn[col].fillna(0, inplace=True)
tst[col].fillna(0, inplace=True)
trn[col] = trn[col].astype(int)
tst[col] = tst[col].astype(int)

col = 'ind_actividad_cliente'
trn[col].fillna(-1, inplace=True)
tst[col].fillna(-1, inplace=True)
trn[col] = trn[col].astype(int)
tst[col] = tst[col].astype(int)

col = 'renta'
trn[col].fillna(0, inplace=True)
tst[col].fillna(0, inplace=True)
# 결측값을 의미하는 '         NA'를 0으로 변환
nan_value = '         NA'
trn[col].replace(nan_value, 0, inplace=True)
tst[col].replace(nan_value, 0, inplace=True)
# trn 데이터를 정수형 (int)로 변환
trn[col] = trn[col].astype(float).astype(int)
tst[col] = tst[col].astype(float).astype(int)


# 불필요한 변수 제거
col = 'tipodom'
trn.drop([col], axis=1, inplace=True)
tst.drop([col], axis=1, inplace=True)

ind_empleado
pais_residencia
sexo
tiprel_1mes
indresi
indext
conyuemp
canal_entrada
indfall
nomprov
segmento


In [13]:
# 코드 1-23. 전처리 수행한 Training Data 확인하기

trn.iloc[:, :23].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 23 columns):
fecha_dato               datetime64[ns]
ncodpers                 int64
ind_empleado             int64
pais_residencia          int64
sexo                     int64
age                      int64
fecha_alta               datetime64[ns]
ind_nuevo                int64
antiguedad               int64
indrel                   int64
ult_fec_cli_1t           datetime64[ns]
indrel_1mes              int64
tiprel_1mes              int64
indresi                  int64
indext                   int64
conyuemp                 int64
canal_entrada            int64
indfall                  int64
cod_prov                 int64
nomprov                  int64
ind_actividad_cliente    int64
renta                    int64
segmento                 int64
dtypes: datetime64[ns](3), int64(20)
memory usage: 2.3 GB


In [14]:
# 코드 1-24. 전처리 완료한 데이터 저장하기 

trn.iloc[:, :23].to_csv('data/train_preprocess.csv', index=False)